# Solar Envelope

In this workshop we will learn how to compute the solar envelop of the building based on a voxelized building envelope.

### 0. Initialization
Importing all necessary libraries and specifying the inputs

In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
from ladybug.sunpath import Sunpath

In [2]:
vs = 3
unit = [vs, vs, vs]
envelope_path = os.path.relpath('../data/optional_envelope.obj')
context_path = os.path.relpath('../data/immediate_context.obj')

In [3]:
# load the mesh from file
envelope_mesh = tm.load(envelope_path)
context_mesh = tm.load(context_path)

# Check if the mesh is watertight
print(envelope_mesh.is_watertight)

True


In [4]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# Visualize the mesh using pyvista plotter
#######

# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the meshes
p.add_mesh(tri_to_pv(envelope_mesh), color='#abd8ff')
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(911.927305702112, 874.9174903521119, 870.154989702112),
 (59.703559999999996, 22.69374465, 17.931243999999992),
 (0.0, 0.0, 1.0)]

In [5]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/voxelized_envelope.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)

In [6]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# Visualize the mesh using pyvista plotter
#######

# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(912.1326102576459, 873.6227949076458, 870.3602942576458),
 (59.703559999999996, 21.19374465, 17.931243999999992),
 (0.0, 0.0, 1.0)]

In [6]:
# initiate sunpath
sp = Sunpath(longitude=4.3571, latitude=52.0116)

# define sun hours : A list of hours of the year for each sun vector
# there are 8760 hours in a year, so the following integers refer to specific hours throughout the year
hoys = []
sun_vectors = []
hour_multiples = 13
for i in range(8760):
    if i%hour_multiples==0:
        # compute the sun object
        sun = sp.calculate_sun_from_hoy(i)
        # extract the sun vector
        sun_vector = sun.sun_vector.to_array()
        # apparantly, if the Z component of sun vector is positive, it is under the horizon 
        if sun_vector[2] < 0.0:
            hoys.append(i)
            sun_vectors.append(sun_vector)

print(np.array(sun_vectors).shape)

(4460, 3)


In [16]:
sun_dir = -np.array(sun_vectors[0:10])
vox_src = envelope_lattice.centroids
ray_dir = np.tile(sun_dir, [len(vox_src),1])
ray_src = np.tile(vox_src, [1, len(sun_dir)]).reshape(-1, 3)
print(ray_src.shape)

(13900, 3)


In [17]:
tri_id, ray_id = context_mesh.ray.intersects_id(ray_origins=ray_src, ray_directions=ray_dir, multiple_hits=False)

In [21]:
hits = np.full((len(ray_dir)), 0)
hits[ray_id] = 1
hits = hits.reshape(len(vox_src), -1)
voxel_hits = np.sum(hits, axis=1)

array([0, 0, 0, ..., 0, 0, 2])

### Credits

In [ ]:
__author__ = "Shervin Azadi and Pirouz Nourian"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Solar Envelope"